# NCBI PubMed Literature Search Strategy & CSV Generation
This notebook allows you to test different keyword strategies for the NCBI API (using PubMed database)

Obtain your API key through your account (optional): https://account.ncbi.nlm.nih.gov/settings/

Edit the `groups` and `logic` in the next code cell, then run the subsequent cells to see the results


In [1]:
import subprocess
import sys

try:
    import requests
    print("'requests' is already installed.")
except ModuleNotFoundError:
    print("'requests' not found. Installing now...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "requests"])
    import requests
    print("'requests' has been installed successfully.")


'requests' is already installed.


# 1. Setup & Define Your Folders and API Key

Make sure your .env file is setup with your API key, and output folders. API key for the NCBI API is optional but recommended. You can find a .env.example in 

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

# Load .env variables
load_dotenv(find_dotenv())

# Get folder paths and API key from environment variables
csv_folder = os.getenv("CSV_FOLDER")
summary_folder = os.getenv("SUMMARY_FOLDER")
api_key = os.getenv("YOUR_NCBI_API_KEY")

# This will create the folders (and parents) if they do NOT exist—does nothing if they do
os.makedirs(csv_folder, exist_ok=True)
os.makedirs(summary_folder, exist_ok=True)

# Check that folders and API key are set
missing = []
if not api_key or api_key == "YOUR_NCBI_API_KEY" or api_key == "your_actual_ncbi_key_here":
    missing.append("API key")
if not os.path.isdir(csv_folder):
    missing.append("CSV folder")
if not os.path.isdir(summary_folder):
    missing.append("Summary folder")

if missing:
    print(f"⚠️ WARNING: Please check the following: {', '.join(missing)}")
else:
    print("✅ Output folders and API key are set up and ready.")

✅ Output folders and API key are set up and ready.


# 2. Test and adjust your keyword strategy

The below 4 sections will help test different keyword groups and their combinations.
- 2.1 Run to define groups of keywords and your exclusion keyword group using AND/OR rules, then define a combination logic
- 2.2. Run to see the number of results returned for each keyword group and the combined query
- 2.3. Run to see the first 10 titles for each keyword group
- 2.4. Run to see the first 10 titles for the combined keyword group

In [ ]:
# === EDIT THIS CELL TO CHANGE YEAR RANGE ===
year_range = "2016:3000[dp]"

# === EDIT THIS CELL TO CHANGE KEYWORDS/LOGIC ===
groups = {
    'group1': 'keyword OR keyword',
    'group2': 'keyword OR keyword',
    'group3': 'keyword',
    'group4': 'keyword',
    'excluded': 'NOT (keyword OR keyword)',
}

logic = "({group1}) AND ({group2}) AND ({group3}) AND ({group4}) {excluded}"
combined_query = logic.format(**groups)

print(f"Keyword groups and logic defined.\nYear filter: {year_range}")
print("Combined PubMed query:", combined_query)

Keyword groups and logic defined.
Year filter: 2016:3000[dp]
Combined PubMed query: (neuroimaging pipeline OR MRI processing OR neuroinformatics pipeline) AND (structural MRI OR T1-weighted OR T2-weighted OR low-field MRI OR portable MRI) AND (continuous integration OR continuous deployment OR CI/CD OR containerization OR containerisation OR version control OR cloud-based OR serverless OR distributed storage OR BIDS OR flywheel.io OR github OR gitlab OR reproducibility) AND (brain OR neuroimaging) NOT (fMRI OR EEG OR MEG OR functional connectivity OR clinical trial)


In [10]:
# === SECTION: Run API Query and Return Total Results for Each Group and Combined Query ===

import requests
import xml.etree.ElementTree as ET

def run_pubmed_query(query, max_results=0, year_range="2016:3000[dp]"):
    """Run PubMed API query and return count/PMIDs/titles"""
    query_with_year = f"{query} AND {year_range}"
    params = {
        'db': 'pubmed',
        'term': query_with_year,
        'retmax': max_results or 1,
        'retmode': 'xml'
    }
    esearch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    esearch_resp = requests.get(esearch_url, params=params)
    root = ET.fromstring(esearch_resp.content)
    count = int(root.findtext('.//Count', '0'))
    pmids = [id_elem.text for id_elem in root.findall('.//Id')]
    titles = []
    if max_results > 0 and pmids:
        efetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
        efetch_params = {'db': 'pubmed', 'id': ','.join(pmids), 'retmode': 'xml'}
        efetch_resp = requests.get(efetch_url, params=efetch_params)
        articles = ET.fromstring(efetch_resp.content).findall('.//PubmedArticle')
        for article in articles:
            title = article.findtext('.//ArticleTitle', default='').replace('\n', ' ').strip()
            titles.append(title)
    return count, pmids, titles

# Print year filter info
print(f"Year limit applied to all queries: {year_range}\n")

# Test individual keyword groups
print("="*50)
print("INDIVIDUAL GROUP RESULTS:")
print("="*50)
for name, query in groups.items():
    if name != 'excluded':  # Skip exclusion group
        count, _, _ = run_pubmed_query(query)
        print(f"{name.upper():<25}: {count} results")

# Print combined query
print("\n" + "="*50)
print("COMBINED LOGIC RESULTS:")
print("="*50)
print(f"Logic: {logic}\n")
print(f"Combined query: {combined_query}")
combined_count, pmids, _ = run_pubmed_query(combined_query)
print(f"Combined results: {combined_count}")

Year limit applied to all queries: 2016:3000[dp]

INDIVIDUAL GROUP RESULTS:
GROUP1                   : 62505 results
GROUP2                   : 75448 results
GROUP3                   : 363979 results
GROUP4                   : 925422 results

COMBINED LOGIC RESULTS:
Logic: ({group1}) AND ({group2}) AND ({group3}) AND ({group4}) {excluded}

Combined query: (neuroimaging pipeline OR MRI processing OR neuroinformatics pipeline) AND (structural MRI OR T1-weighted OR T2-weighted OR low-field MRI OR portable MRI) AND (continuous integration OR continuous deployment OR CI/CD OR containerization OR containerisation OR version control OR cloud-based OR serverless OR distributed storage OR BIDS OR flywheel.io OR github OR gitlab OR reproducibility) AND (brain OR neuroimaging) NOT (fMRI OR EEG OR MEG OR functional connectivity OR clinical trial)
Combined results: 51


The next block will show the first 10 titles for each keyword group (except the excluded keyword group).

Based on this, you can go back and adust your keyword groups.

In [5]:
# === SECTION: Print First 10 Titles for Each Group (Excluding 'excluded') ===

max_titles = 10  # Number of titles to print for each group

for name, query in groups.items():
    if name != 'excluded':
        print("\n" + "="*50)
        print(f"FIRST {max_titles} TITLES FOR GROUP: {name.upper()}")
        print("="*50)
        count, pmids, titles = run_pubmed_query(query, max_results=max_titles, year_range=year_range)
        if titles:
            for i, title in enumerate(titles, 1):
                print(f"{i}. {title}")
        else:
            print("No titles found for this group.")



FIRST 10 TITLES FOR GROUP: GROUP1
1. Comparison of two preprocessing methods for
2. Mediating role of feeder connections in the relationship between brain network efficiency and episodic memory in amnestic MCI.
3. Breaking data silos: incorporating the DICOM imaging standard into the OMOP CDM to enable multimodal research.
4. Real-time color flow mapping of ultrasound microrobots.
5. An Automatic 3D PET Tumor Segmentation Framework Assisted by Geodesic Sequences.
6. High-resolution 3D whole-heart bright- and black-blood imaging with co-registered T2 mapping at 0.55 T.
7. Higher-order Sonification of the Human Brain.
8. Mitigating Organophosphate Nerve Agent, Soman (GD)-Induced Long-Term Neurotoxicity: Saracatinib, a Src Tyrosine Kinase Inhibitor, as a Potential Countermeasure.
9. Frequency of autoimmune-associated exogenous psychoses in routine clinical care.
10. GABA, Glx, and GSH in the cerebellum: their role in motor performance and learning across age groups.

FIRST 10 TITLES FOR 

The next block will show the first 10 titles for the combined query.

Based on the results you can go back and adjust your groups and logic.

In [6]:
# === SECTION: Show First 10 Titles for Combined Keyword Group ===

max_titles_to_print = 10 # <-- Adjust this if you want more or fewer

# Fetch results and titles
combined_count, _, _ = run_pubmed_query(combined_query, max_results=0, year_range=year_range)

if combined_count > 0:
    print("\n" + "="*50)
    print(f"FIRST {max_titles_to_print} TITLES FOR COMBINED QUERY:")
    print("="*50)
    _, _, titles = run_pubmed_query(combined_query, max_results=max_titles_to_print, year_range=year_range)
    for i, title in enumerate(titles, 1):
        print(f"{i}. {title}")
else:
    print("\nNo results found for the combined query.")



FIRST 10 TITLES FOR COMBINED QUERY:
1. Region-based U-nets for fast, accurate, and scalable deep brain segmentation: Application to Parkinson Plus Syndromes.
2. Normative range of MRI-derived fetal brain volume throughout gestation: a prospective study.
3. fMRIPrep Lifespan: Extending A Robust Pipeline for Functional MRI Preprocessing to Developmental Neuroimaging.
4. Recent advances in the detection and management of motor dysfunction in Alzheimer's disease.
5. Evaluating the Quality of Brain MRI Generators.
6. Modular strategies for spatial mapping of diverse cell type data of the mouse brain.
7. FastSurfer-LIT: Lesion inpainting tool for whole-brain MRI segmentation with tumors, cavities, and abnormalities.
8. Host genetics maps to behaviour and brain structure in developmental mice.
9. A New Tool for Extracting Static and Dynamic Parameters from [
10. Quantitative reliability assessment of brain MRI volumetric measurements in type II GM1 gangliosidosis patients.


# 2. Export PubMed Results to CSV

The below script will use your combined query to download titles and abstracts and save them to a CSV file, including author name, title, abstract, year and doi. It will also update the summary table to include the total of found and downloaded records, the source the final query and a timestamp for record keeping purposes.

In [11]:
# === SECTION: Download CSV and Update Summary ===

# === SECTION: IMPORTS ===
import os
import csv
import requests
import xml.etree.ElementTree as ET
from datetime import datetime

# === SECTION: UTILITY FUNCTION: Get Next CSV Name ===
def get_next_csv_name(folder, base_name):
    i = 1
    while True:
        csv_name = f"{base_name}_v{i}.csv"
        csv_path = os.path.join(folder, csv_name)
        if not os.path.exists(csv_path):
            return f"{base_name}_v{i}", csv_path  # returns base name for summary and full path
        i += 1

# === SECTION: MAIN FUNCTION: Download PubMed Results to CSV ===
def download_ncbi_to_csv(query, csv_folder, api_key, year_range, max_results=None):
    os.makedirs(csv_folder, exist_ok=True)
    esearch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    efetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"

    query_with_year = f"{query} AND {year_range}"
    params = {
        'db': 'pubmed',
        'term': query_with_year,
        'retmax': max_results if max_results is not None else 10000,
        'retmode': 'xml',
        'api_key': api_key
    }
    print(f"Querying PubMed for: {query_with_year}")
    esearch_resp = requests.get(esearch_url, params=params)
    root = ET.fromstring(esearch_resp.content)
    ids = [id_elem.text for id_elem in root.findall('.//Id')]
    print(f"Retrieved {len(ids)} PubMed IDs")

    base_name, csv_path = get_next_csv_name(csv_folder, "pubmed")
    print(f"Writing results to CSV: {csv_path}")

    count_downloaded = 0
    with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['first_author', 'all_authors', 'title', 'abstract', 'year', 'doi'])

        if ids:
            efetch_params = {
                'db': 'pubmed',
                'id': ','.join(ids),
                'retmode': 'xml',
                'api_key': api_key
            }
            efetch_resp = requests.get(efetch_url, params=efetch_params)
            root = ET.fromstring(efetch_resp.content)

            for article in root.findall('.//PubmedArticle'):
                authors = []
                first_author = ''
                author_list = article.find('.//AuthorList')
                if author_list is not None:
                    for i, author in enumerate(author_list.findall('Author')):
                        last = author.findtext('LastName', default='')
                        fore = author.findtext('ForeName', default='')
                        full_name = f"{fore} {last}".strip()
                        if full_name:
                            authors.append(full_name)
                        if i == 0 and full_name:
                            first_author = full_name
                all_authors = '; '.join(authors)

                title = article.findtext('.//ArticleTitle', default='').replace('\n', ' ').strip()

                abstract = ''
                abstract_elem = article.find('.//Abstract')
                if abstract_elem is not None:
                    abstract = ' '.join([abst.text for abst in abstract_elem.findall('AbstractText') if abst.text]).replace('\n', ' ').strip()

                year = ''
                pubdate = article.find('.//Journal/JournalIssue/PubDate/Year')
                if pubdate is not None:
                    year = pubdate.text
                else:
                    year = article.findtext('.//Article/Journal/JournalIssue/PubDate/Year', default='')
                year = year.strip() if year else ''

                doi = article.findtext('.//ArticleId[@IdType="doi"]', default='')

                writer.writerow([first_author, all_authors, title, abstract, year, doi])
                count_downloaded += 1

    print(f"Downloaded {count_downloaded} records to {csv_path}")
    # Capture timestamp at end of download in ISO format without seconds
    timestamp = datetime.now().strftime("%Y-%m-%dT%H:%M")
    return len(ids), count_downloaded, base_name, timestamp  # <--- return timestamp

# === SECTION: SUMMARY ROW APPEND WITH TIMESTAMP ===
def append_summary_row(summary_folder, base_name, found, downloaded, query, timestamp):
    summary_csv_path = os.path.join(summary_folder, "summary_csv.csv")
    os.makedirs(summary_folder, exist_ok=True)
    header = "source,found,downloaded,query combination,timestamp\n"
    row = f"{base_name},{found},{downloaded},\"{query}\",{timestamp}\n"

    # Check if file exists and if it's empty
    file_exists = os.path.exists(summary_csv_path)
    is_empty = not file_exists or os.path.getsize(summary_csv_path) == 0

    with open(summary_csv_path, 'a', encoding='utf-8', newline='') as f:
        if is_empty:
            f.write(header)
        f.write(row)
    print(f"Summary row added for {base_name}")

# === SECTION: RUN DOWNLOAD AND SUMMARY ===
found, downloaded, base_name, timestamp = download_ncbi_to_csv(
    combined_query,
    csv_folder,
    api_key,
    year_range=year_range,
    max_results=None  # Change to None to download all results
)

append_summary_row(
    summary_folder,
    base_name,
    found=found,
    downloaded=downloaded,
    query=combined_query,
    timestamp=timestamp
)


Querying PubMed for: (neuroimaging pipeline OR MRI processing OR neuroinformatics pipeline) AND (structural MRI OR T1-weighted OR T2-weighted OR low-field MRI OR portable MRI) AND (continuous integration OR continuous deployment OR CI/CD OR containerization OR containerisation OR version control OR cloud-based OR serverless OR distributed storage OR BIDS OR flywheel.io OR github OR gitlab OR reproducibility) AND (brain OR neuroimaging) NOT (fMRI OR EEG OR MEG OR functional connectivity OR clinical trial) AND 2016:3000[dp]
Retrieved 51 PubMed IDs
Writing results to CSV: C:/Users/petra/Documents/UniKCL/Workshop/csvs/pubmed_v1.csv
Downloaded 51 records to C:/Users/petra/Documents/UniKCL/Workshop/csvs/pubmed_v1.csv
Summary row added for pubmed_v1
